In [ ]:
# Utils packages
import ast
from datetime import datetime
from pathlib import Path, PurePath
from typing import List

# Visualization packages
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from PIL import Image

# Analysis packages
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

# Model packages
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

## Read dataset file

In [ ]:
DATA_PATH = "data" # Directory to drop collected data.

In [ ]:
hentais_df = pd.read_csv(
    Path(DATA_PATH) / "preprocessed" / "metadata.csv",
    index_col=None,
    converters={
        column_name: ast.literal_eval
        for column_name in ["tag", "group", "parody", "character", "artist", "category", "image_urls"]
    }
)
hentais_df

## Data preparation

The following preparation steps aim to create training and validation sets that can be used for machine learning.
For example, if a hentai tag is rare, we will remove it from the target variable.
The model will not learn how to predict that genre if the data covering it is insufficient.

In [ ]:
# Get label frequencies in descending order
label_freq = hentais_df["tag"].explode().value_counts()
label_freq = label_freq / len(hentais_df["tag"])
label_freq = label_freq.head(70)

# Bar plot
style.use("fivethirtyeight")
plt.figure(figsize=(12, 16))
sns.barplot(y=label_freq.index.values, x=label_freq, order=label_freq.index)
plt.title("Labels", fontsize=14)
plt.xlabel("Frequency")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

## Remove infrequent labels

We will consider as a rare label every label that is covered by less than 5% in our dataset.
We will assume that rare labels are very hard to predict due to lack of sufficient data.
The model that we will train later will not focus on predicting these labels.
So, we need to make some transformation in the label column (tag) where we ignore infrequent labels by hiding them.

In [ ]:
rare_label_df = label_freq[label_freq < 0.05]
rare_label_df

Transform Genre into a list of labels and remove the rare ones

In [ ]:
hentais_df["tag"] = hentais_df["tag"].apply(lambda x: [l for l in x if l not in rare_label_df])
hentais_df

In [ ]:
plt.figure(figsize=(14, 4))
hentais_df["num_favorites"].plot.hist(ax=plt.subplot(1, 2, 1), bins=100, title="Favorites")
hentais_df["num_pages"].plot.hist(ax=plt.subplot(1, 2, 2), bins=100, title="Pages")
plt.show()

We need to complete the full path to locate training and test images from the current working directory.

In [ ]:
def hentai_filename(hentai: pd.Series):
    return str(PurePath(DATA_PATH) / "preprocessed" / str(hentai["id"]) / Path(hentai["image_url"]).name)

hentais_df = hentais_df.explode("image_urls", ignore_index=True).rename(columns={"image_urls": "image_url"})
hentais_df = hentais_df.assign(filename=hentais_df.apply(hentai_filename, axis=1))
hentais_df

## Remove non-JPG images

For practical reasons, we will stick with JPG images for the moment.

In [ ]:
hentais_df = hentais_df[hentais_df["filename"].str.endswith(".jpg")]

## Image examples

Let's display some examples of training images.

In [ ]:
nobjs = 8 # Maximum number of images to display
ncols = 4 # Number of columns in display
nrows = nobjs // ncols # Number of rows in display

plt.figure(figsize=(14, 4 * nrows))
for i, filename in enumerate(hentais_df["filename"].sample(nrows * ncols)):
    ax = plt.subplot(nrows, ncols, i + 1)
    ax.imshow(Image.open(filename).convert("RGB"))

## Train / val split

Splitting the modeling data into training and validation is common in machine learning practice.
We will be allocating 80% of the images for training and 20% for validation.
Usually, we keep a final test set to communicate performance results but we will not really need it in this notebook.

In [ ]:
X, Y = hentais_df["filename"].to_numpy(), hentais_df["tag"].to_numpy()
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=44)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=44)

print("Number of hentais for training: ", len(X_train))
print("Number of hentais for validation: ", len(X_val))
print("Number of hentais for test: ", len(X_test))

## Label encoding

The original targets are lists of strings that can be easily understood by humans.
But, if we want to build and train a neural network we need to create binary labels (multi-hot encoding).
This is critical for multi-label classification.

In order to binarize our labels, we will be using scikit-learn's MultiLabelBinarizer.

In [ ]:
# Fit the multi-label binarizer on the training set
mlb = MultiLabelBinarizer()
mlb.fit(y_train)

# Analyze the number of labels in the dataset for
# the output layer of our subsequent model.
labels_df = pd.Series(mlb.classes_)
labels_df

Analyze the number of labels in the dataset for the output layer of our subsequent model.

In [ ]:
nlabels = len(labels_df)

transform the targets of the training and test sets

In [ ]:
y_train_bin = mlb.transform(y_train)
y_val_bin = mlb.transform(y_val)
y_test_bin = mlb.transform(y_test)

## Input pipeline

If you are familiar with keras.preprocessing you may know the image data iterators (E.g. ImageDataGenerator, DirectoryIterator).
These iterators are convenient for multi-class classfication where the image directory contains one subdirectory for each class.
But, in the case of multi-label classification, having an image directory that respects this structure is not possible because one observation can belong to multiple classes at the same time.

That is where the tf.data API has the upper hand.
- It is faster.
- It provides fine-grained control.
- It is well integrated with the rest of TensorFlow.

We first need to write some function to parse image files and generate a tensor representing the features and a tensor representing the labels.
- In this function we can resize the image to adapt to the input expected by the model.
- We can also normalize the pixel values to be between 0 and 1. This is a common practice that helps speed up the convergence of training.

If we consider every pixel as a feature, we would like these features to have a similar range so that the gradients don't go out of control and that we only need one global learning rate multiplier.

In [ ]:
IMG_SIZE = 224 # Specify height and width of image to match the input format of the model
CHANNELS = 3 # Keep RGB color channels to match the input format of the model

In [ ]:
def parse_function(filename: tf.Tensor, label: int) -> (tf.Tensor, List[int]):
    # Read an image from a file
    image_string = tf.io.read_file(filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_image(image_string, channels=CHANNELS, expand_animations=False)
    # Resize it to fixed shape
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image_resized / 255.0
    return image_normalized, label

To train a model on our dataset we want the data to be:

- Well shuffled
- Batched
- Batches to be available as soon as possible.

These features can be easily added using the tf.data.Dataset abstraction.

In [ ]:
BATCH_SIZE = 256 # Big enough to measure an F1-score
AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically
SHUFFLE_BUFFER_SIZE = 1024 # Shuffle the training data by a chunck of 1024 observations

AUTOTUNE will adapt the preprocessing and prefetching workload to model training and batch consumption.
The number of elements to prefetch should be equal to (or possibly greater than) the number of batches consumed by a single training step.
AUTOTUNE will prompt the tf.data runtime to tune the value dynamically at runtime and reduce GPU and CPU idle time.

We can now create a function that generates training and validation datasets for TensorFlow.

In [ ]:
def create_dataset(filenames: List[str], labels: List[int], is_training: bool=True) -> tf.data.Dataset:
    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    # Parse and preprocess observations in parallel
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)

    if is_training == True:
        # This is a small dataset, only load it once, and keep it in memory.
        dataset = dataset.cache()
        # Shuffle the data each buffer size
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)

    # Batch the data for multiple steps
    dataset = dataset.batch(BATCH_SIZE)
    # Fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [ ]:
train_ds = create_dataset(X_train, y_train_bin)
val_ds = create_dataset(X_val, y_val_bin)

Each batch will be a pair of arrays (one that holds the features and another one that holds the labels).
The features array will be of shape (BATCH_SIZE, IMG_SIZE, IMG_SIZE, CHANNELS).
The labels array will be of shape (BATCH_SIZE, N_LABELS) where N_LABELS is the maximum number of labels.
Let's verify the shapes of these arrays by analyzing the first batch:

In [ ]:
for f, l in train_ds.take(1):
    print("Shape of features array:", f.numpy().shape)
    print("Shape of labels array:", l.numpy().shape)

## Transfert Learning

In [ ]:
feature_extractor_layer = hub.KerasLayer(
    "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4",
    input_shape=(IMG_SIZE, IMG_SIZE, CHANNELS)
)

The feature extractor accepts images of shape (224, 224, 3) and returns a 1280-length vector for each image.

We should freeze the variables in the feature extractor layer, so that the training only modifies the new classification layers.
Usually, it is a good practice when working with datasets that are very small compared to the orginal dataset the feature extractor was trained on.

In [ ]:
feature_extractor_layer.trainable = False

## Main Model

In [ ]:
model = keras.Sequential([
    feature_extractor_layer,
    layers.Dense(1024, activation="relu"),
    layers.Dense(nlabels, activation="sigmoid")
])
model.summary()

## Model training and evaluation

In [ ]:
LR = 1e-5 # Keep it small when transfer learning
EPOCHS = 30

Compile the model to configure the training process.

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
  loss="binary_crossentropy",
  metrics=["accuracy"]
)

Now, we pass the training dataset of (features, labels) to fit the model and indicate a seperate dataset for validation.
The performance on the validation dataset will be measured after each epoch.

In [ ]:
output_dir = Path("job")
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
savedmodel_dir = output_dir / "export" / "savedmodel"
model_export_path = savedmodel_dir / timestamp
checkpoint_path = output_dir / "checkpoints"
tensorboard_path = output_dir / "tensorboard"

In [ ]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(),
        keras.callbacks.EarlyStopping(patience=2),
        keras.callbacks.TensorBoard(str(tensorboard_path)),
        keras.callbacks.ModelCheckpoint(str(checkpoint_path)),
    ]
)

In [ ]:
plt.figure(figsize=(14, 4))

ax = plt.subplot(1, 2, 1)
ax.plot(history.history["accuracy"])
ax.plot(history.history["val_accuracy"])
ax.title("model accuracy")
ax.ylabel("accuracy")
ax.xlabel("epoch")
ax.legend(["train", "validation"])

ax = plt.subplot(1, 2, 2)
ax.plot(history.history["loss"])
ax.plot(history.history["val_loss"])
ax.title("model loss")
ax.ylabel("loss")
ax.xlabel("epoch")
ax.legend(["train", "validation"])
plt.show()